## In this notebook, we preprocess data by removing html tag, special character, meaningless word, ...

- Result: Preprocessed Data csv file (but stil contains null data, we continue do this next notebook)

In [1]:
import pandas as pd
import numpy as np

import lxml.html
import re
from bs4 import BeautifulSoup

## Reading files

In [2]:
df_master = pd.read_csv('Extract_incidents.csv')

In [3]:
df_master.head()

,Number,Priority,Incident area,Opened,Closed,Status,Problem,Configuration item,Short description,Description,Assignment group,Resolution notes,Reassignment count,Resolve time
0,INC0284448,P2 - High,SAP :: Sonda Procwork (PRW),44013.002951,44024.875023,Closed,NaN,NaN,VF01 - Not working,"Please, the transaction VF01 is not working, w...",IMT APS SAP Order to Cash L2,"Hello Daniel,\n\nMany thanks for your reply, a...",1.0,158300.0
1,INC0284449,P4 - Low,SAP :: Technical,44013.003183,44023.083368,Closed,NaN,SAP-ECC-PRD,Enqueue Processing,Alert Details:\n Name....................Enque...,IMT TCS SAP Basis,Monitoring the lock entries in P01 via inciden...,0.0,5694.0
2,INC0180368,P2 - High,Infrastructure & Servers :: Server (physical/v...,43831.011447,43842.500012,Closed,NaN,GOYCSM0001DC,GOYCSM0001DC.ds.givaudan.com (Server does not ...,"Dear Team,\nWe received a ticket from our moni...",IMT NLI Ops EAME,There was a planned maintenance onsite that wa...,1.0,126566.0
3,INC0180373,P3 - Moderate,SAP :: Order to Cash (OTC),43831.132951,43871.625023,Closed,NaN,NaN,I cant connect to ESKER PRD & UAT,"I already got access for ESKER QA, UAT & PRD. ...",IMT Desktop Infra Support Indonesia,User confirmed issue was solved,3.0,1919058.0
4,INC0180380,P3 - Moderate,Business Application :: Google,43831.405255,43841.416678,Closed,NaN,NaN,I cannot access my google drive shortcut from ...,I cannot access my google drive shortcut from ...,IMT Service Desk AME,"User G drive was inactive, i reconfigured it, ...",0.0,128.0


## Preprocess

In [4]:
df = df_master.copy()

In [5]:
df.columns = df.columns.str.replace(' ','_')

In [6]:
description = df['Description']

### Detect and remove html tags

In [7]:
def detect_html_tags(row):
    if row is None:
        return False
    if re.search("<[^<]+?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});", str(row)):
        return True
#     return lxml.html.fromstring(str(row)).find('.//*') is not None
    return False

In [8]:
def remove_html_tags(row):
    return re.sub(re.compile('<[^<]+?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});'), '', str(row))

In [9]:
def remove_html_tags_v2(row):
    return BeautifulSoup(str(row), "lxml").text

In [10]:
def html_removal(df: pd.DataFrame) -> pd.DataFrame:
    first_removal = df.apply(remove_html_tags)
    remained_html_tags = first_removal.apply(detect_html_tags)
    remain_df = df[remained_html_tags]
    second_removal = remain_df.apply(remove_html_tags_v2)
    first_removal[remained_html_tags] = second_removal
    
    return first_removal

### Remove Chinese words

In [11]:
def detect_chinese(row):
    if str(row) == '':
        return False
    if re.search("[\u4e00-\u9FFF]", str(row)):
        return True
    return False

In [12]:
def remove_chinese(row):
    regex = re.compile('[^\u0020-\u024F]')
    return regex.sub('', str(row))

def chinese_removal(df: pd.DataFrame) -> pd.DataFrame:
    df_final = df.apply(remove_chinese) 
    return df_final

### Numerical Removal

In [39]:
def numerical_removal(df: pd.DataFrame) -> pd.DataFrame:
    return df.str.replace('\d+', '')

### Detect special characters

In [90]:
def special_detection(row):
    try:
        str(row).encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [92]:
df['Description'].apply(special_detection).value_counts()

True     111239
False      6005
Name: Description, dtype: int64

### Check

In [13]:
columns = ['Description', 'Short_description', 'Resolution_notes']

In [24]:
# Remove html tags
for column in columns:
    df[column] = html_removal(df[column])

In [25]:
# Check
for column in columns:
    print("Html tags check: \n", df[column].apply(detect_html_tags).value_counts())
    print("=====")

Html tags check: 
 False    117244
True          1
Name: Description, dtype: int64
=====
Html tags check: 
 False    117245
Name: Short_description, dtype: int64
=====
Html tags check: 
 False    117245
Name: Resolution_notes, dtype: int64
=====


In [30]:
html_tags = df['Description'].apply(detect_html_tags)
df[html_tags]

,Number,Priority,Incident_area,Opened,Closed,Status,Problem,Configuration_item,Short_description,Description,Assignment_group,Resolution_notes,Reassignment_count,Resolve_time
10604,INC0207504,P4 - Low,Business Application :: OpenText,43879.547245,43897.625023,Closed,NaN,GVECSM4802AD,Error moving content on Archive Server,See the error message below and the attached d...,IMT TCS Document Management,We did disabled particular setting in AC admin...,1.0,694768.0


In [16]:
for column in columns:
    df[column] = chinese_removal(df[column])

In [17]:
for column in columns:
    print("Html tags check: \n", df[column].apply(detect_chinese).value_counts())
    print("=====")

Html tags check: 
 False    117245
Name: Description, dtype: int64
=====
Html tags check: 
 False    117245
Name: Short_description, dtype: int64
=====
Html tags check: 
 False    117245
Name: Resolution_notes, dtype: int64
=====


In [35]:
df.drop(df[df.Number == "INC0207504"].index, inplace=True)b

In [42]:
for column in columns:
    print("Html tags check: \n", df[column].apply(detect_html_tags).value_counts())
    print("=====")

Html tags check: 
 False    117244
Name: Description, dtype: int64
=====
Html tags check: 
 False    117244
Name: Short_description, dtype: int64
=====
Html tags check: 
 False    117244
Name: Resolution_notes, dtype: int64
=====


In [93]:
# Remove numerical words 
for column in columns:
    df[column] = numerical_removal(df[column])

<ipython-input-39-581d5757fa09>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  return df.str.replace('\d+', '')


In [95]:
# Remove rows contains special characters

In [103]:
for column in columns:
    special_rows = df[column].apply(special_detection)
    df[column] = df[column][special_rows]

In [104]:
df.shape

(117244, 14)

## Export file

In [106]:
df.to_csv('Preprocessed_Data.csv', index=False)